In [1]:
import sys
import geopandas as gpd
from shutil import copyfile, move

sys.path.insert(0, './HLB')
import hlb_utils

In [5]:
grid = gpd.read_file('../VECTOR/teste_clip/teste_grid.geojson')
copas = gpd.read_file('../VECTOR/teste_clip/teste_copas.geojson')

In [6]:
grid

,geometry
0,"POLYGON ((294194.638 7516266.401, 294194.638 7..."


In [7]:
copas

,geometry
0,"POLYGON ((294187.240 7516276.024, 294188.269 7..."
1,"POLYGON ((294179.425 7516270.129, 294181.413 7..."
2,"POLYGON ((294192.691 7516272.905, 294194.713 7..."
3,"POLYGON ((294188.680 7516282.023, 294190.154 7..."


In [9]:
gpd.clip(copas, grid, keep_geom_type=True).explode().reset_index()

,level_0,level_1,geometry
0,0,0,"POLYGON ((294187.240 7516276.024, 294188.269 7..."
1,1,0,"POLYGON ((294181.838 7516271.425, 294182.579 7..."
2,1,1,"POLYGON ((294181.838 7516267.427, 294182.030 7..."
3,2,0,"POLYGON ((294192.691 7516272.905, 294194.638 7..."


In [2]:
pre_processing = hlb_utils.PreProcessing('../IMGS/odm_orthophoto.tif', '../VECTOR/copas_setor.geojson')

In [3]:
pre_processing.split_img(patch_size=256, overlap=25, save_img=True)

Cortando imagens: 100%|██████████| 986/986 [00:30<00:00, 31.82it/s]


Finished!
Total images:  118


In [4]:
pre_processing.split_vector_2()

Salvando arquivos:   2%|▏         | 2/118 [00:05<05:06,  2.65s/it]


ValueError: Length of names must match number of levels in MultiIndex.

In [ ]:
pre_processing.copas

In [ ]:
pre_processing.grid.drop(['id'], inplace=True, axis=1)

In [ ]:
pre_processing.copas.reset_index(inplace=True)
pre_processing.grid.reset_index(inplace=True)

In [ ]:
pre_processing.grid

In [ ]:
gpd.clip(pre_processing.grid.loc[[5]], pre_processing.copas, keep_geom_type=True).to_file('../Mask_RCNN/datasets/clip5_keep.geojson', driver='GeoJSON')

In [ ]:
pre_processing.grid.loc[[32]]

### Make annotation

In [ ]:
from PIL import Image
for i in id_test:
    move('../IMGS/CORTES/{:05}.tif'.format(i), '../Mask_RCNN/datasets/canopy/real_test/images/TIF/test_2/{:05}.tif'.format(i))
    


In [ ]:
outfile = os.path.splitext(os.path.join(outpath, name))[0] + ".jpg"
try:
    im = Image.open(os.path.join(root, name))
    print("Generating jpeg for {}".format(name))
    im.thumbnail(im.size)
    im.convert('RGB').save(outfile, "JPEG", quality=100)
except Exception as e:
    print(e)

In [ ]:
grid = gpd.read_file('../VECTOR/grid_from_img.geojson')

In [ ]:
grid.insert(1, 'split', '')

In [ ]:
grid

In [ ]:
grid['split'] = 'test'

In [ ]:
type(grid)

In [ ]:
None == None

In [ ]:
import os
import pandas as pd
shape_path_aux = '../Mask_RCNN/datasets/real_test/talhao_over55/results'

result_paths = []
for filename in os.listdir(shape_path_aux):
    if os.path.splitext(filename)[1].lower() == '.geojson' and \
            os.path.splitext(filename)[0].lower() != 'test_union':
        result_paths.append(os.path.join(shape_path_aux, filename))

pols = pd.concat([gpd.read_file(i) for i in result_paths], axis=0).reset_index(drop=True)
pols['ID'] = pols.index
pols.insert(1, column='remove', value='')

delete_lines = []
for i in pols.index:
    if pols.loc[i, 'remove'] == '':
        polygon = pols.loc[i, 'geometry']
        intersec = pols.overlaps(polygon)
        intersec = intersec[intersec]
        list_result = list(intersec.index)

        if len(list_result) != 0:
            maior = pols.loc[list_result, 'geometry'].area.idxmax(axis=0)
#             menor = pols.loc[list_result, 'geometry'].area.idxmin(axis=0)

            for z in [i for i in list_result if i != maior]:
                inter_poly = pols.loc[z, 'geometry'].intersection(pols.loc[maior, 'geometry'])
                if inter_poly.geom_type == 'Polygon' and inter_poly.area > 0.5:
                    delete_lines.append(z)
                    
                    pols.at[z, 'remove'] = 'remove'

#     if pols.loc[i, 'union'] == 0:
#         pols.at[i, 'union'] = i

# pols = pols.dissolve(by='union')
pols.to_file(os.path.join(shape_path_aux,'test_union_2.geojson'), driver='GeoJSON')